<a href="https://colab.research.google.com/github/UmutSonmezz/Langchain-Excel-Reader-RAG/blob/main/Langchain_excel_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
#!curl -X POST http://localhost:11434/api/generate
import subprocess
subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


<Popen: returncode: None args: ['ollama', 'serve']>

In [ ]:
!ollama pull llama3.1

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕▏ 4.7 GB                         
pulling 11ce4ee3e170... 100% ▕▏ 1.7 KB                         
pulling 0ba8f0e314b4... 100% ▕▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕▏   96 B                         
pulling 1a4c3c319823... 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [ ]:
!pip install -q -U langchain
!pip install -q -U pypdf
!pip install -q -U pandas
!pip install -q -U  langchain-llms-ollama
!pip install -q -U  langchain_community
!pip install -q -U  python-dotenv

ERROR: Could not find a version that satisfies the requirement langchain-llms-ollama (from versions: none)
ERROR: No matching distribution found for langchain-llms-ollama


In [ ]:
!pip install accelerate
#!pip install -i https://pypi.org/simple/ bitsandbytes
#!pip uninstall bitsandbytes
!pip install bitsandbytes
!pip install  transformers
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
import os
from langchain_community.llms import Ollama
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()
from langchain.chains import ConversationalRetrievalChain
import nest_asyncio
nest_asyncio.apply()

import torch


In [ ]:
!pip install -q -U faiss-gpu
!pip install -q -U sentence-transformers

In [ ]:
llm = Ollama(model="llama3.1", temperature=0.1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/QNA_EXCEL.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from langchain.schema.document import Document
text = ""
for ind in df.index:
    text += f"{df['qna_id'][ind]}  {df['question'][ind]}  {df['answer'][ind]} {df['tags'][ind]} {df['query_1'][ind]} {df['query_2'][ind]} {df['query_3'][ind]} {df['query_4'][ind]} {df['query_5'][ind]} {df['query_6'][ind]} {df['query_7'][ind]} {df['query_8'][ind]} {df['query_9'][ind]} {df['query_10'][ind]} {df['query_11'][ind]} {df['query_12'][ind]} {df['query_13'][ind]} {df['query_14'][ind]} {df['query_15'][ind]} {df['query_16'][ind]} {df['query_17'][ind]} {df['query_18'][ind]} {df['query_19'][ind]} {df['query_20'][ind]}\n#####\n"
    documents = Document(page_content=text, metadata={"source": "local"})

In [ ]:
!pip install -q -U sentence-transformers==2.7.0

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators="[#####]")
chunked_docs  = text_splitter.split_documents([documents])
embeddings = HuggingFaceEmbeddings()
#cusstom metadata customize metadata

In [ ]:
!pip install faiss-gpu

In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))


# Connect query to FAISS index using a retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)


In [ ]:
qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever,return_source_documents=True)

In [ ]:
import sys
chat_history = []
query = input('Prompt: ')
result = qa_chain.invoke({'question': query, 'chat_history': chat_history})
print('Answer: ' + result['answer'] + '\n')
chat_history.append((query, result['answer']))

Prompt:  Anadolu mobilden giriş yaptım cep telefonundan ePub kitapları neden açamıyorum?
Answer: Mobil cihazınıza uygun ePub okuyucu programlardan birini indirerek ePub kitabı açabilirsiniz.

